<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.



# Описание проекта

Необходимо создать модель, которая будет способна по заданному комментарию определить, является ли он токсичным. Токсичность может быть и в позитивных и в негативных комментариях, например, содержать ненормативную лексику. Такие комментрарии необходимо модерировать.

# Цель проекта

Создать модель, выяляющую токсичность комментариев пользователей на работу интернет - магазина.

# Задачи ислледования

1. Считать и проверить исходные данные<br>
2. Подготовить исходные данные к обработке алгортимом обучения<br>
3. Выполнить обучение модели несколькими алгоритмами на тренировочной выбрке и выбрать лучший<br>
4. Провести тестирование выбранной модели на тестовой выборке<br>

# Описание данных

Данные находятся в файле toxic_comments.csv.<br> 
*text* -текст комментария<br>
*toxic* — целевой признак.

**Основные шаги:**

Часть 1. Изучение общей информации:

1. Загрузка библиотек, считывание данных.
2. Изученеие файлов с данными, получение общей информации
3. Нахождение и ликвидация пропусков.

Часть 2. Подготовка данных:

1. Приведение данных к нужным типам.
2. Нахождение и исправление аномалий и ошибок.
3. Проверка баланса данных
4. Очистка и лемматизация
5. Подготовка корпуса данных для тренировочной и тестовой выборок
6. Создание tf-idf для выборок

Часть 3. Обучение моделей

1. LogisticRegression
2. XGBClassifier
3. LGBMClassifier

Часть 4. Анализ моделей и вывод

1. Проанализировать полученные результаты
2. Сделать общий вывод и рекомендовать модель

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import os

In [2]:
try:
    data = pd.read_csv('datasets/toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропуски отсутствуют

Удаление неинформативного столбца **Unnamed:**

In [5]:
data = data.drop(['Unnamed: 0'],axis=1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Для поиска правильного POS-тега для каждого слова выполняется его сопоставление с правильным входным символом, который принимает WordnetLemmatizer, и передача в качестве второго аргумента в lemmatize().
В nltk для этого есть метод nltk.pos_tag(). Он принимает список слов, а возвращает кортеж с тегом POS. Ключевым моментом здесь является сопоставление POS-тегов NLTK с форматом, принятым лемматизатором wordnet.

Воспользуемся готовыми и напишем функции для лемматизации текста:<br>
get_wordnet_pos(word) - возвращает POS тэг для слова(к какой части речи относится слово)<br>
clear_text(text) - приводит к нижнему регистру и очишает от символов, кроме букв английского алфавита(через re)<br>
lemmatize(text) - проводит лемматизацию текста<br>

In [11]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [12]:
#Функция лемматизации текста
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lemm_list])      
    return lemm_text

In [13]:
#Функция очистки текста
def clear_text(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = clear_text.lower().split()
    return ' '.join(clear_text)

Проверка работы функций на одной строке

In [14]:
data.loc[0, 'text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [15]:
lemmatize(clear_text(data.loc[1, 'text']))

'd aww he match this background colour i m seemingly stuck with thanks talk january utc'

#### Лемматизация

В файле after_lemm.csv будет сохранен датафрейм после лемматизации в целях его применения в дальнейшей работе

In [16]:
fal = 'after_lemm.csv'

При первом запуске лемматизация заняла 1ч 45 мин

%%time<br>
data['lemm_text'] = data['text'].apply(lambda text: lemmatize(clear_text(text)))<br>
<br>
CPU times: total: 1h 45min 38s<br>
Wall time: 1h 45min 40s<br>

Если ноутбук запускается не в первый раз, то считываемс сразу сохраненный датафрейм с лемматизированным столбцом

In [17]:
if os.path.isfile(fal):
    data = pd.read_csv(fal)
else:
    %%time
    data['lemm_text'] = data['text'].apply(lambda text: lemmatize(clear_text(text)))

In [18]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [19]:
try:
    data = data.drop(['Unnamed: 0'],axis=1)
except:
    pass

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159292 non-null  object
 1   toxic      159292 non-null  int64 
 2   lemm_text  159281 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


Если лемматизация проводится впервые, то записываем результат в файл

In [21]:
if not(os.path.isfile(fal)):
    print('Запись файла')
    data.to_csv('after_lemm.csv')

#### Проверка баланса классов

In [22]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Нетоксичных комментариев почти в 10 раз больше. Этот факт необходимо будет учитывать при обучении

#### Разделение на выборки

features - признаки<br>
target - целевой признак

train - тренировочная<br>
valid - валидационная

In [23]:
train_features, valid_features, train_target, valid_target = train_test_split(
    data['lemm_text'], data['toxic'], test_size=0.2, random_state=12345)

In [24]:
train_features.shape, valid_features.shape, train_target.shape, valid_target.shape

((127433,), (31859,), (127433,), (31859,))

valid - валидационная<br>
test - тестовая

In [25]:
valid_features, test_features, valid_target, test_target = train_test_split(
    valid_features, valid_target, test_size=0.5, random_state=12345)

In [26]:
valid_features.shape, test_features.shape, valid_target.shape, test_target.shape

((15929,), (15930,), (15929,), (15930,))

In [27]:
%%time
train_corpus = train_features.values.astype('U')
valid_corpus = valid_features.values.astype('U')
test_corpus = test_features.values.astype('U')

CPU times: total: 609 ms
Wall time: 610 ms


Поиск стоп-слов<br>
(пакет stopwords модуля nltk.corpus библиотеки nltk)

In [28]:
stop_words = list(set(stopwords.words('english')))

#### Вычисление TF-IDF для корпуса текстов

Создание счётчика со стоп-словами

In [29]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

**.fit_transform()** для обучающей выборки и **.transform()** для валидационной и тестовой:

In [30]:
train_tf_idf = count_tf_idf.fit_transform(train_corpus)
valid_tf_idf = count_tf_idf.transform(valid_corpus)
test_tf_idf = count_tf_idf.transform(test_corpus)

In [31]:
train_tf_idf.shape

(127433, 132168)

In [32]:
valid_tf_idf.shape

(15929, 132168)

In [33]:
test_tf_idf.shape

(15930, 132168)

**Функция для определение метрики качества создаваемой модели**

In [34]:
def scoring(model,tf_idf,target,selection='валидационная'):
    prediction = model.predict(tf_idf)
    score_f1 = f1_score(target, prediction)
    
    print(f'Выборка - {selection}. F1 - {score_f1:.2f}')

#### Вывод 1

1) Данные не имеют пропусков, дубликатов
2) Удален неинформативный столбец в исходных данных
3) Проведена чистка и лемматизация исходного признака. Для повторной загрузки датафрейм сохранен в отдельный файл
4) В данных присутствовует дисбаланс целевого признака
5) тексты преобразованы в векторный вид (NLTK библиотека)
6) TF-IDF увеличил количество признаков, равное количеству в мешке слов.

## Обучение

Модели для обучения:
* LogisticRegression
* XGBClassifier
* LGBMClassifier


Функция определения лучших гиперпараметров моделей градиентного бустинга, обучения модели на полченных гиперпараметрах и определения лучшего значения score

In [35]:
def learn_model(model,param_grid):
    """
    Функция определения лучших гиперпараметров моделей градиентного бустинга, 
    обучения модели на полченных гиперпараметрах и определения лучшего значения score
    """
        
    
    # Получение гиперпараметров исходной модели
    dir_params = model.get_params()
    # Подбор гиперпараметров
    grid_cv = GridSearchCV(model, param_grid, scoring='f1')
    
    start_time = datetime.now()
    grid_cv.fit(train_tf_idf, train_target)
    time_grid_search_cv = datetime.now() - start_time
    
    print()
    print('Лучшие гиперпараметры')
    print(grid_cv.best_score_)
    print(grid_cv.best_params_)
    
    # Замена значений гиперпараметров на лучшие значения
    for k,v in grid_cv.best_params_.items():
        dir_params[k] = v
    
     # Применение новых гиперпараметров к модели
    model = model.set_params(**dir_params)
   
    # Обучение модели на новых гиперпараметрах
    start_time = datetime.now()
    model.fit(train_tf_idf, train_target)
    time_model_fit = datetime.now() - start_time
    # Получение предсказания
    start_time = datetime.now()
        
    prediction = model.predict(valid_tf_idf)
    score_f1 = f1_score(valid_target, prediction)
    
    print(f'F1 на валидационной выборке - {score_f1:.2f}')
      
    
    time_pred_valid = datetime.now() - start_time
    
    
    return model, score_f1,time_grid_search_cv,time_model_fit,time_pred_valid

Функция добавления полученных результатов в итоговую таблицу

In [36]:
def app_data_model_param(df,list_param):
    df.loc[-1] = list_param
    df.index = df.index + 1
    df = df.sort_index()

Таблица для сбора данных полученных результатов

In [37]:
model_param = pd.DataFrame(columns=['model','f1','time_grid','time_fit_model','time_predict','pointer'])

In [38]:
model_param

,model,f1,time_grid,time_fit_model,time_predict,pointer


### LogisticRegression

In [39]:
name_model = 'LogisticRegression'

In [40]:
model = LogisticRegression(random_state=12345, class_weight='balanced')

In [41]:
param_grid = {'C': np.linspace(0.0001, 100, 10)}

In [42]:
%%time
model,f1,time_grid_search_cv,time_model_fit,time_pred_valid = learn_model(model,param_grid)


Лучшие гиперпараметры
0.7547523379005472
{'C': 11.1112}
F1 на валидационной выборке - 0.76
CPU times: total: 16min 25s
Wall time: 2min 9s


In [43]:
app_data_model_param(model_param,
                     [name_model,
                      f1,
                      time_grid_search_cv.microseconds,
                      time_model_fit.microseconds,
                      time_pred_valid.microseconds,
                      model
                     ]
                    )

### Модель градиентного бустинга XGBClassifier

In [45]:
name_model = 'XGBClassifier'

In [46]:
model = XGBClassifier(random_state=12345, n_jobs=-1)

In [47]:
param_grid  = {'max_depth': [6, 8],
               'n_estimators': [300, 400]}

In [48]:
%%time
model,f1,time_grid_search_cv,time_model_fit,time_pred_valid = learn_model(model,param_grid)


Лучшие гиперпараметры
0.7637068446786681
{'max_depth': 8, 'n_estimators': 400}
F1 на валидационной выборке - 0.77
CPU times: total: 2h 6min 1s
Wall time: 16min 4s


In [49]:
app_data_model_param(model_param,
                     [name_model,
                      f1,
                      time_grid_search_cv.microseconds,
                      time_model_fit.microseconds,
                      time_pred_valid.microseconds,
                      model
                     ]
                    )

### Модель градиентного бустинга LGBMClassifier

In [51]:
name_model = 'LGBMClassifier'

In [52]:
model = LGBMClassifier(n_jobs=-1, random_state=12345, class_weight='balanced')

In [53]:
param_grid = {'max_depth': [6,8],
              'learning_rate': [0.05, 0.1],
              'n_estimators': [200,300]}

In [54]:
%%time
model,f1,time_grid_search_cv,time_model_fit,time_pred_valid = learn_model(model,param_grid)


Лучшие гиперпараметры
0.7469366441335888
{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 300}
F1 на валидационной выборке - 0.76
CPU times: total: 1h 30min 7s
Wall time: 11min 55s


In [55]:
app_data_model_param(model_param,
                     [name_model,
                      f1,
                      time_grid_search_cv.microseconds,
                      time_model_fit.microseconds,
                      time_pred_valid.microseconds,
                      model
                     ]
                    )

**Итоговая таблица¶**

In [56]:
model_param = model_param.reset_index()

In [57]:
model_param = model_param.drop(['index'],axis=1)

In [58]:
model_param = model_param.sort_values('f1',ascending=False)

In [59]:
model_param

,model,f1,time_grid,time_fit_model,time_predict,pointer
1,XGBClassifier,0.769177,147335,81889,62485,"XGBClassifier(base_score=None, booster=None, c..."
0,LogisticRegression,0.764974,800114,959092,4986,"LogisticRegression(C=11.1112, class_weight='ba..."
2,LGBMClassifier,0.758060,471327,386154,79787,"LGBMClassifier(class_weight='balanced', max_de..."


### Вывод 2:

По результатам обучения и проверки моделей на валидационной выборке лучшей моделью стала:

In [60]:
model_param[model_param.f1 == model_param.f1.max()].iloc[0,0]

'XGBClassifier'

Лучшей по времени обучения 

In [62]:
model_param[model_param.time_fit_model == model_param.time_fit_model.min()].iloc[0,0]

'XGBClassifier'

Почти все модели показали значение f1 в районе 0.76.<br>

## Выводы

**Итоговая таблица¶**

In [63]:
model_param

,model,f1,time_grid,time_fit_model,time_predict,pointer
1,XGBClassifier,0.769177,147335,81889,62485,"XGBClassifier(base_score=None, booster=None, c..."
0,LogisticRegression,0.764974,800114,959092,4986,"LogisticRegression(C=11.1112, class_weight='ba..."
2,LGBMClassifier,0.758060,471327,386154,79787,"LGBMClassifier(class_weight='balanced', max_de..."


#### Проверка лучшей модели на тестовой выборке

In [64]:
best_model = model_param[model_param.f1 == model_param.f1.max()].iloc[0,5]

In [65]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=12345, ...)

In [66]:
pred_test = best_model.predict(test_tf_idf)

In [67]:
score_f1 = f1_score(test_target, pred_test)
print(f'F1 на тестовой выборке - {score_f1:.2f}')

F1 на тестовой выборке - 0.77


В целом все модели дают хорошие показатели метрики, но отличаются по скорости обучения.

Лучшей моделью, рекомендованной заказчику является **XGBClassifier**

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны